In [1]:
import os
import sys
import glob

import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.backend.tensorflow_backend import set_session

import matplotlib.pyplot as plt

import nibabel as nib

Using TensorFlow backend.


In [2]:
def load_nii(path):
    nii = nib.load(path)
    vol = nii.get_data().astype(np.float32)
    return np.squeeze(vol)

In [3]:
run_dirs = glob.glob('../runs/*/predict')
run_dirs = sorted(run_dirs)

print('found {} runs'.format(len(run_dirs)), '\n')

for i, run in enumerate(run_dirs):
    print(i, run, '\n')

found 3 runs 

0 ../runs/vae_20190417_1843_gpu=6_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_lr=0.0001_pl=100_is=0.01_vr=1.0_lw=[1_1]_tag=brains_test/predict 

1 ../runs/vae_20190520_0944_gpu=3_bs=16_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_lr=0.0001_pl=100_is=0.01_vr=0.5_lw=[1_1]_tag=voxelmorph_s1s2s3s4_s0/predict 

2 ../runs/vae_20190520_2040_gpu=6_bs=16_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_lr=0.0001_pl=100_is=0.01_vr=0.5_lw=[1_1]_tag=split5_s1s2s3s4_s0_full_size/predict 



In [4]:
# set run dir
run_dir = run_dirs[2]

pre_dirs = glob.glob(os.path.join(glob.escape(run_dir), '*', '*'))
pre_dirs = sorted(pre_dirs)

print('found {} pre_dirs'.format(len(pre_dirs)))

for i, pre_dir in enumerate(pre_dirs):
    #print(i, os.path.basename(pre_dir))
    print(i, pre_dir.split('/')[-2] + '/' + pre_dir.split('/')[-1])

found 2 pre_dirs
0 split_test/gen_040_04
1 split_test/gen_056_04


In [5]:
# generator config, run_dir, etc...
pre_dir = pre_dirs[1]

csv_path = os.path.join(pre_dir, 'meta.csv')
csv = pd.read_csv(csv_path)

print('found meta.csv: {}'.format(csv.shape))

found meta.csv: (271, 40)


In [6]:
# classifier
#clf_dir = '../../dx/runs/clf_20190516_2031_gpu=4_bs=32_lr=0.0001_b1=0.9_b2=0.999_ep=0.01_bn=True_mp=True_lk=0.0_ls=[16_2_32_1_32_2_64_1_64_2_128_1_128_2_256_1_256_1]_tag=split5_s2s4_s1s3/'
clf_dir = '../../dx/runs/clf_20190520_1945_gpu=6_bs=32_lr=0.0001_b1=0.9_b2=0.999_ep=0.0001_bn=True_mp=True_lk=0.0_ls=[16_2_32_1_32_2_64_1_64_2_128_1_128_2_256_1_256_1]_tag=split5_full_size_s1s2s3s4_s0/'

model_name = 'clf_0010'
model_file = os.path.join(clf_dir, model_name+'.h5')

if os.path.isfile(model_file):
    print('found model file')
else:
    print('model file not found!')

found model file


In [7]:
# gpu config
gpu_id = 6

if gpu_id is not None:                                                     
    gpu = '/gpu:' + str(gpu_id)                                            
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)                       
    config = tf.ConfigProto()                                              
    config.gpu_options.allow_growth = True                                 
    config.allow_soft_placement = True                                     
    set_session(tf.Session(config=config))                                 
else:                                                                      
    gpu = '/cpu:0'

In [9]:
print('starting predict')

with tf.device(gpu):
    
    # load classifier model
    classifier = keras.models.load_model(model_file)
    
    for i, (_, row) in enumerate(csv.iterrows()):
        
        if i % 10 == 0:
            print('step', i)
        
        xr = load_nii(row['img_path_0'])[None, ..., None]
        yr = load_nii(row['img_path_1'])[None, ..., None]
        yf = load_nii(row['img_path_yf'])[None, ..., None]
        
        batch = np.concatenate([xr, yr, yf], axis=0)
        
        pred = classifier.predict([batch])
        
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_xr'] = pred[0, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yr'] = pred[1, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yf'] = pred[2, 1]
        
print('finished predict')

csv_out_path = os.path.join(pre_dir, model_name+'.csv')

print('writing csv to {}'.format(csv_out_path))

csv.to_csv(csv_out_path, index=False)

starting predict
step 0
step 10
step 20
step 30
step 40
step 50
step 60
step 70
step 80
step 90
step 100
step 110
step 120
step 130
step 140
step 150
step 160
step 170
step 180
step 190
step 200
step 210
step 220
step 230
step 240
step 250
step 260
step 270
finished predict
writing csv to ../runs/vae_20190520_2040_gpu=6_bs=16_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_lr=0.0001_pl=100_is=0.01_vr=0.5_lw=[1_1]_tag=split5_s1s2s3s4_s0_full_size/predict/split_test/gen_056_04/clf_0010.csv


In [10]:
clf_csvs = glob.glob(os.path.join(glob.escape(pre_dir), '[!meta]*.csv'))
clf_csvs = sorted(clf_csvs)

for i, clf_csv in enumerate(clf_csvs):
    print(i, os.path.basename(clf_csv))

0 clf_0010.csv


In [11]:
csv_path = clf_csvs[0]
csv = pd.read_csv(csv_path)
print(csv.shape)

(271, 43)


In [12]:
scsv = csv[csv.pat_dx_1 == 2]
print('s: xr -> yr:', (scsv.pAD_yr - scsv.pAD_xr).mean())
print('s: xr -> yf:', (scsv.pAD_yf - scsv.pAD_xr).mean())

pcsv = csv[csv.pat_dx_1 == 3]
print('p: xr -> yr:', (pcsv.pAD_yr - pcsv.pAD_xr).mean())
print('p: xr -> yf:', (pcsv.pAD_yf - pcsv.pAD_xr).mean())

s: xr -> yr: 0.1159980022641815
s: xr -> yf: 0.441847338427016
p: xr -> yr: 0.3181100513124318
p: xr -> yf: 0.4023435180234554


In [13]:
print(csv[['pAD_xr', 'pAD_yr', 'pAD_yf', 'pat_dx_1']].mean())
csv[['pAD_xr', 'pAD_yr', 'pAD_yf', 'pat_dx_1']].head(10)*100

pAD_xr      0.249418
pAD_yr      0.438504
pAD_yf      0.676980
pat_dx_1    2.361624
dtype: float64


,pAD_xr,pAD_yr,pAD_yf,pat_dx_1
0,10.920704,0.572314,26.667410,200.0
1,11.128416,99.982774,99.966133,200.0
2,0.228174,15.027128,3.106096,200.0
3,1.820767,0.575360,14.456289,200.0
4,0.683231,0.079784,6.820168,200.0
5,0.304578,3.004045,32.802531,300.0
6,2.534587,5.247945,68.943632,300.0
7,48.008946,30.106309,99.223000,300.0
8,92.148787,95.489311,99.799913,300.0
9,0.100285,0.094518,62.060314,200.0


In [14]:
def get_metric(csv_in, col, t, metric):

    true_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 3)).sum()
    true_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 2)).sum()
    false_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 2)).sum()
    false_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 3)).sum()
    
    true = true_pos + true_neg
    false = false_pos + false_neg

    if metric == 'f1':
        prec = true_pos / (true_pos + false_pos)
        reca = true_pos / (true_pos + false_neg)

        f1_score = 2 * (prec * reca) / (prec + reca)
        m = f1_score
        
    elif metric == 'acc':
        acc_p = true_pos / (true_pos + false_neg)
        acc_s = true_neg / (true_neg + false_pos)

        balanced_acc = (acc_p + acc_s) / 2
        m = balanced_acc
    
    return (round(m, 3), true, false)

In [15]:
def get_best_threshold(csv_in, col, metric):
    l = []
    
    for t in np.arange(0, 1, 0.01):
        m = get_metric(csv_in, col, t, metric)
        l.append((*m, t))
        
    return (max(l)[3])

In [16]:
def get_split_metric(csv_in, col, cv_split, metric):
    
    csv_split = csv_in[csv_in.cv_split == cv_split]
    csv_other = csv_in[csv_in.cv_split != cv_split]
    
    t = get_best_threshold(csv_other, col, metric)
    
    m = get_metric(csv_split, col, t, metric)
    print('split {}: {}, t={}'.format(cv_split, m, t))
    
    return m

In [17]:
def get_cv_metric(csv_in, col, metric):
    l = [get_split_metric(csv_in, col, s, metric)[0] for s in range(5)]
    return np.array(l).mean()

In [18]:
# create 5 folds with equal number of pMCI/sMCI samples each
csv = csv.sample(frac=1).reset_index(drop=True)

pMCI = csv[csv.pat_dx_1 == 3].img_id_0.values
sMCI = csv[csv.pat_dx_1 == 2].img_id_0.values

p_bins = np.linspace(0, len(pMCI), 6).astype(int)[1:-1]
s_bins = np.linspace(0, len(sMCI), 6).astype(int)[1:-1]

p_splits = np.split(pMCI, p_bins)
s_splits = np.split(sMCI, s_bins)

In [19]:
# write folds to csv
for i, (p_split, s_split) in enumerate(zip(p_splits, s_splits)):
    csv.loc[csv.img_id_0.isin(p_split), 'cv_split'] = i
    csv.loc[csv.img_id_0.isin(s_split), 'cv_split'] = i

In [20]:
get_cv_metric(csv, 'pAD_xr', 'f1')

split 0: (0.548, 25, 28), t=0.01
split 1: (0.571, 31, 24), t=0.01
split 2: (0.654, 35, 18), t=0.01
split 3: (0.642, 36, 19), t=0.01
split 4: (0.6, 31, 24), t=0.01


0.603

In [21]:
get_cv_metric(csv, 'pAD_yr', 'f1')

split 0: (0.68, 37, 16), t=0.26
split 1: (0.545, 35, 20), t=0.32
split 2: (0.773, 43, 10), t=0.26
split 3: (0.647, 43, 12), t=0.87
split 4: (0.679, 38, 17), t=0.26


0.6648000000000002

In [22]:
get_cv_metric(csv, 'pAD_yf', 'f1')

split 0: (0.588, 32, 21), t=0.86
split 1: (0.5, 29, 26), t=0.86
split 2: (0.711, 40, 13), t=0.86
split 3: (0.609, 37, 18), t=0.86
split 4: (0.593, 33, 22), t=0.86


0.6002

In [23]:
get_cv_metric(csv, 'pAD_xr', 'acc')

split 0: (0.54, 32, 21), t=0.24
split 1: (0.711, 40, 15), t=0.16
split 2: (0.669, 36, 17), t=0.16
split 3: (0.643, 39, 16), t=0.27
split 4: (0.668, 37, 18), t=0.16


0.6462

In [24]:
get_cv_metric(csv, 'pAD_yr', 'acc')

split 0: (0.741, 37, 16), t=0.26
split 1: (0.629, 35, 20), t=0.32
split 2: (0.83, 43, 10), t=0.26
split 3: (0.732, 43, 12), t=0.87
split 4: (0.704, 41, 14), t=0.87


0.7272000000000001

In [25]:
get_cv_metric(csv, 'pAD_yf', 'acc')

split 0: (0.567, 33, 20), t=0.99
split 1: (0.554, 29, 26), t=0.86
split 2: (0.696, 41, 12), t=0.99
split 3: (0.679, 37, 18), t=0.86
split 4: (0.643, 33, 22), t=0.86


0.6278